# Simple Linear Program

A simple linear program with only 2 variables and 3 constraints in order to learn the API of the OR Tools linear solver.

In [2]:
%reload_ext autoreload
%autoreload 2

import numpy as np # numerical library
import matplotlib.pyplot as plt # plotting library
import utils
import datetime as dt
import pandas as pd

from ortools.linear_solver import pywraplp

In [4]:
# this is the "solver" which does the actual work of solving the linear program
solver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

In [5]:
# declare variables
x = solver.NumVar(0, solver.infinity(), 'x')
y = solver.NumVar(0, solver.infinity(), 'y')    

In [6]:
# Constraint 0: x + 2y <= 14.
constraint0 = solver.Constraint(-solver.infinity(), 14)
constraint0.SetCoefficient(x, 1)
constraint0.SetCoefficient(y, 2)

# Constraint 1: 3x - y >= 0.
constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(x, 3)
constraint1.SetCoefficient(y, -1)

# Constraint 2: x - y <= 2.
constraint2 = solver.Constraint(-solver.infinity(), 2)
constraint2.SetCoefficient(x, 1)
constraint2.SetCoefficient(y, -1)

In [7]:
# Objective function: 3x + 4y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 4)
objective.SetMaximization()

In [8]:
solver.Solve()

0

In [9]:
opt_solution = 3 * x.solution_value() + 4 * y.solution_value()
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
# The value of each variable in the solution.
print('Solution:')
print('x = ', x.solution_value())
print('y = ', y.solution_value())
# The objective value of the solution.
print('Optimal objective value =', opt_solution)

Number of variables = 2
Number of constraints = 3
Solution:
x =  5.999999999999998
y =  3.9999999999999996
Optimal objective value = 33.99999999999999


In [10]:
objective.Value()

33.99999999999999

## Timepoints as Variables

Now that we are familiar with how to set up the variables, constraints, and optimization for a linear program, we write a slightly more complicated linear program that uses *lists* of variables that represent time points.

Translate the following linear program in code:

$$
max_{h_1, \ldots, h_{24}} \sum_{i=1}^{24} h_i
\\
\text{s.t.}~h_i \leq 5~\forall i~\text{and} ~h_5 \leq2
$$

In [11]:
timesolver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

all_hours = []
objective = timesolver.Objective()

for i in range(24):
    h = timesolver.NumVar(-timesolver.infinity(), 5, 'h'+ str(i)) 
    if i == 4:
        h = timesolver.NumVar(-timesolver.infinity(), 2, 'h'+ str(i)) 
    objective.SetCoefficient(h, 1)
    all_hours.append(h)

objective.SetMaximization()
timesolver.Solve()
print(objective.Value())

117.0


In [12]:
for i in range(24):
    print(all_hours[i].solution_value())
    

5.0
5.0
5.0
5.0
2.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0


## Simple linear program with two resources

Using the basic structure of the examples above, we write a linear program that cost-optimizes a portfolio of natural gas and solar resources to match Harbor's historic generation profile.



In [3]:
#Load Harbor historical hourly generation and emissions for 2014-2018. Group by datetime to sum generation and emissions from all units. Filter for specific year -- make sure selected year isn't leap year.
harborgen = utils.get_harbor_data('data/HarborHourly_2014-18.csv')
harborgen = harborgen.groupby(['datetime'])['kwh', 'SO2_MASS..lbs.', 'NOX_MASS..lbs.', 'CO2_MASS..tons.'].sum()

# Create string object that is the chosen year
year = '2018'

# Filter Harbor data by the chosen year, using the string object above.
harborgen.filter(like=year, axis=0)

# Load solar generation profile for 1 KW system. Change all years in solar dataset to match year chosen above. Set datetime as index.
solargen = utils.get_solar_data()
solargen['datetime'] = solargen['datetime'].mask(solargen['datetime'].dt.year == 1900, solargen['datetime'] + pd.offsets.DateOffset(year=int(year)))
solargen.set_index(['datetime'], inplace=True, drop=True)

/Users/AnnieDillon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/AnnieDillon/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:692: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


In [4]:
solargen.head()

,kwh
datetime,
2018-01-01 00:00:00,0.0
2018-01-01 01:00:00,0.0
2018-01-01 02:00:00,0.0
2018-01-01 03:00:00,0.0
2018-01-01 04:00:00,0.0


In [5]:
harborgen.head()

,kwh,SO2_MASS..lbs.,NOX_MASS..lbs.,CO2_MASS..tons.
datetime,,,,
2014-01-01 00:00:00,0.0,0.0,0.0,0.0
2014-01-01 01:00:00,0.0,0.0,0.0,0.0
2014-01-01 02:00:00,0.0,0.0,0.0,0.0
2014-01-01 03:00:00,0.0,0.0,0.0,0.0
2014-01-01 04:00:00,0.0,0.0,0.0,0.0


In [18]:
solargas_solver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# Declare solar and gas nameplate capacity variables.
s_kw = solargas_solver.NumVar(0, solargas_solver.infinity(), 's_kw')
g_kw = solargas_solver.NumVar(0, solargas_solver.infinity(), 'g_kw')


In [19]:
#Keep track of gas generation variable for every hour.
gas_hourly_gen = []

#Introduce objective object so we can refer to it in the for loop.
objective = solargas_solver.Objective()

#Loop through every hour, creating hourly constraints. **Change to datetime index.
for ind in harborgen.index:
    gen = solargas_solver.NumVar(0, solargas_solver.infinity(), 'gen'+ str(ind))
    gas_hourly_gen.append(gen)
    
    #Summed solar and gas generation must be equal or greater to demand in all hours.
    constraint1 = solargas_solver.Constraint(harborgen.loc[ind,'kwh'], solargas_solver.infinity())
    constraint1.SetCoefficient(s_kw, solargen.loc[ind,'kwh'])
    constraint1.SetCoefficient(gen, 1)
    
    #Gas generation must be less than or equal to gas capacity in all hours.
    constraint2 = solargas_solver.Constraint(0, solargas_solver.infinity())
    constraint2.SetCoefficient(g_kw, 1)
    constraint2.SetCoefficient(gen, -1)
   
    # Add cost here for gas variable generation ($/kwh)
    objective.SetCoefficient(gen, .021)

In [20]:
## Add in numbers here for capital and fixed costs -- for fixed, we need to assume a certain plant lifetime and amortize over that period. ******

objective.SetCoefficient(s_kw, 630)
objective.SetCoefficient(g_kw, 672)

objective.SetMinimization()
solargas_solver.Solve()
print("total cost =", objective.Value())

#Sums gas generation for every hour. Prints gas generation for every hour that it is > 0.
summed_gas_gen = 0
for i in range(8760):
    summed_gas_gen = summed_gas_gen + gas_hourly_gen[i].solution_value()
    if gas_hourly_gen[i].solution_value() > 0.0:
        print("gas generation at hour", str(i), gas_hourly_gen[i].solution_value())

print("annual gas generation (kwh) =", summed_gas_gen)
print("solar nameplate capacity (kw) =", s_kw.solution_value())
print("gas nameplate capacity (kw) =", g_kw.solution_value())

total cost = 232972157.88
gas generation at hour 55 6840.0
gas generation at hour 58 22319.999999999996
gas generation at hour 59 17999.999999999996
gas generation at hour 62 2430.0
gas generation at hour 63 64679.999999999985
gas generation at hour 64 8640.0
gas generation at hour 103 6460.0
gas generation at hour 104 62719.999999999985
gas generation at hour 105 16379.999999999998
gas generation at hour 175 15400.000000000002
gas generation at hour 176 7920.000000000001
gas generation at hour 177 8320.0
gas generation at hour 178 12000.0
gas generation at hour 179 340.00000000000006
gas generation at hour 199 10829.999999999998
gas generation at hour 200 11999.999999999998
gas generation at hour 201 169.99999999999997
gas generation at hour 202 16799.999999999996
gas generation at hour 203 8370.0
gas generation at hour 223 5600.0
gas generation at hour 224 42750.0
gas generation at hour 225 41650.0
gas generation at hour 226 22440.0
gas generation at hour 227 14040.000000000004
gas g